In [1]:
solfeggio = [
    174,
    285,
    396,
    417,
    528,
    639,
    741,
    852,
    963
]

In [2]:
_1_2 = list(map(lambda x: x/2, solfeggio))
_1_4 = list(map(lambda x: x/4, solfeggio))
_1_8 = list(map(lambda x: x/8, solfeggio))
_1_16 = list(map(lambda x: x/16, solfeggio))
_1_32 = list(map(lambda x: x/32, solfeggio))
print(_1_2)
print(_1_4)
print(_1_8)
print(_1_16)
print(_1_32)

[87.0, 142.5, 198.0, 208.5, 264.0, 319.5, 370.5, 426.0, 481.5]
[43.5, 71.25, 99.0, 104.25, 132.0, 159.75, 185.25, 213.0, 240.75]
[21.75, 35.625, 49.5, 52.125, 66.0, 79.875, 92.625, 106.5, 120.375]
[10.875, 17.8125, 24.75, 26.0625, 33.0, 39.9375, 46.3125, 53.25, 60.1875]
[5.4375, 8.90625, 12.375, 13.03125, 16.5, 19.96875, 23.15625, 26.625, 30.09375]


In [3]:
""" bliss
342.462856 Hz
335.17311 Hz
337.957545 Hz
330.66780 Hz
11.795056 Hz
7.289745 Hz
4.50531 Hz
2.784435 Hz
"""

' bliss\n342.462856 Hz\n335.17311 Hz\n337.957545 Hz\n330.66780 Hz\n11.795056 Hz\n7.289745 Hz\n4.50531 Hz\n2.784435 Hz\n'

In [4]:
print(342.462856 - (342.462856 - 330.66780) / 2)
print(330.66780 + (342.462856 - 330.66780) / 2)
print(337.957545 - (337.957545 - 335.17311) / 2)
print(335.17311 + (337.957545 - 335.17311) / 2)

336.565328
336.565328
336.56532749999997
336.56532749999997


In [23]:
astral = (
    # change this to the frequency you want to use, for example 528
    # 336.565328 
    # 342.4914324
    # 432.081216
    
    # 174
    # 285
    # 396
    # 417
    # 528
    # 639
    # 741
    852
    # 963
    
         )

a = 11.795056 / 2
front_left = astral + a
back_right = astral - a

a = 2.784435 / 2
front_right = astral + a
back_left = astral - a

print(front_left,front_right)
print(back_left,back_right)

857.897528 853.3922175
850.6077825 846.102472


In [24]:
import numpy as np

x = {
    'front_left':front_left,'front_right':front_right,
    'back_left':back_left,'back_right':back_right
}

print(x['front_left']-x['front_right'])
print(x['back_left']-x['back_right'])

print(x['front_left']-x['back_left'])
print(x['front_right']-x['back_right'])

print(x['front_left']-x['back_right'])
print(x['front_right']-x['back_left'])

4.5053104999999505
4.5053104999999505
7.289745499999981
7.289745499999981
11.795055999999931
2.7844350000000304


### Warning: 

You will need at least 10 GB of free memory for 1 hour at 96000 sample rate and 32 bit int

In [25]:
import numpy as np
from pydub import AudioSegment
import io
import os

home = os.path.expanduser('~')


# Define the frequencies for the binaural beat
# l_1 = 528  # Frequency for left ear
# l_2 = 639  # Frequency for left ear
# r_1 = 741 # Frequency for right ear
# r_2 = 852  # Frequency for right ear

l_1 = front_left  # Frequency for left ear
r_1 = front_right  # Frequency for right ear
l_2 = back_left  # Frequency for left ear
r_2 = back_right  # Frequency for right ear


# Duration and sample rate of the binaural beat
duration = 60 * 60  # Duration in seconds
sample_rate = 96000  # Sample rate in Hz 
chunk_size = 48000  # Size of each chunk (1 second)

# Create an empty BytesIO object to store the audio data
audio_data = io.BytesIO()

# Process the audio in chunks
for start in range(0, duration * sample_rate, chunk_size):
    end = start + chunk_size
    t = np.linspace(start / sample_rate, end / sample_rate, chunk_size, endpoint=False)

    # Generate sinusoidal waveforms for each frequency
    left_wave1 = np.sin(2 * np.pi * l_1 * t)
    right_wave1 = np.sin(2 * np.pi * r_1 * t)
    left_wave2 = np.sin(2 * np.pi * l_2 * t)
    right_wave2 = np.sin(2 * np.pi * r_2 * t)

    # Combine the left and right waveforms to create the binaural beat
    binaural_left = left_wave1 + left_wave2
    binaural_right = right_wave1 + right_wave2

    # Normalize the amplitudes
    binaural_left /= np.max(np.abs(binaural_left))
    binaural_right /= np.max(np.abs(binaural_right))

    # Combine left and right channels
    stereo_audio = np.column_stack((binaural_left, binaural_right))

    # Convert stereo audio to PyDub AudioSegment
    max_amplitude = 2**31 - 1  # Maximum amplitude for 32-bit integer encoding
    stereo_audio_scaled = (stereo_audio * max_amplitude).astype(np.int32)
    stereo_audio_segment = AudioSegment(
        stereo_audio_scaled.tobytes(),
        frame_rate=sample_rate,
        sample_width=4,  # 32-bit encoding
        channels=2  # Stereo
    )

    # Append the chunk to the audio_data BytesIO object
    audio_data.write(stereo_audio_segment.raw_data)

# Create a new AudioSegment from the audio_data BytesIO object
audio_segment = AudioSegment(
    audio_data.getvalue(),
    frame_rate=sample_rate,
    sample_width=4,
    channels=2
)

# Save the binaural beat as an AIFF file
audio_segment.export(home + "/Music/binaural_beat.aiff", format="aiff")
print("Binaural beat saved as 'binaural_beat.aiff'.")

Binaural beat saved as 'binaural_beat.aiff'.
